## Step1：Import necessary packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Step2：Import dataset

In [2]:
df = pd.read_csv('diabetes.csv')
df.head(10)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   
5            5      116             74              0        0  25.6   
6            3       78             50             32       88  31.0   
7           10      115              0              0        0  35.3   
8            2      197             70             45      543  30.5   
9            8      125             96              0        0   0.0   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
5                     0.201   30        0  
6                     0.248   26        1  
7                     0.134   29        0  
8                     0.158   53        1  
9                     0.232   54        1

In [3]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

## Step3: Data preprocessing
### Step3.1: Figure out null value

In [4]:
df.replace({'Pregnancies':0,
            'Glucose':0,
            'BloodPressure':0,
            'SkinThickness':0,
            'Insulin':0,
            'BMI':0,
            'DiabetesPedigreeFunction':0,
            'Age':0},np.nan,inplace=True)

In [5]:
df.isna().sum()

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

### Step3.2: Using mode to fill these null value

In [7]:
df['Pregnancies'] = df['Pregnancies'].fillna(df['Pregnancies'].mode()[0])
df['Glucose'] = df['Glucose'].fillna(df['Glucose'].mode()[0])
df['BloodPressure'] = df['BloodPressure'].fillna(df['BloodPressure'].mode()[0])
df['SkinThickness'] = df['SkinThickness'].fillna(df['SkinThickness'].mode()[0])
df['Insulin'] = df['Insulin'].fillna(df['Insulin'].mode()[0])
df['BMI'] = df['BMI'].fillna(df['BMI'].mode()[0])

df['Pregnancies'] = df['Pregnancies'].astype('int')
df['Glucose'] = df['Glucose'].astype('int')
df['SkinThickness'] = df['SkinThickness'].astype('int')
df['Insulin'] = df['Insulin'].astype('int')
df['BloodPressure'] = df['BloodPressure'].astype('int')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
Pregnancies                 768 non-null int64
Glucose                     768 non-null int64
BloodPressure               768 non-null int64
SkinThickness               768 non-null int64
Insulin                     768 non-null int64
BMI                         768 non-null float64
DiabetesPedigreeFunction    768 non-null float64
Age                         768 non-null int64
Outcome                     768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


## Step4: Extract features
### Step4.1: Exclude outliners

In [9]:
Features_with_outliers=['Pregnancies', 'BloodPressure', 'SkinThickness', 'Insulin','BMI',
                        'DiabetesPedigreeFunction','Age']
def remove_outliers_iqr(data):
    
    # Calculate the first quartile (Q1) and third quartile (Q3)
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    
    # Calculate the interquartile range (IQR)
    IQR = Q3 - Q1
    
    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Remove outliers
    data = np.where(data>upper_bound, upper_bound, np.where(data<lower_bound,lower_bound,data))
   
    return data[(data >= lower_bound) & (data <= upper_bound)]

In [10]:
for column in Features_with_outliers:
    df[column] = remove_outliers_iqr(df[column])

## Step5: Build Model
### Step5.1: Dividing data into train set and test set

In [11]:
X = df.drop(columns=['Outcome'])  # Drop the 'target' column to get the features
Y = df['Outcome']  # Scelect only the 'target' column as the target variable

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = 0.2, random_state = 5) 

### Step5.2: Training model with data

In [14]:
from sklearn.linear_model import LogisticRegression
regressor = LogisticRegression()
regressor = regressor.fit(X_train, Y_train)

/home/lance/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Step5.3: Predicting using trained model

In [15]:
Y_pred = regressor.predict(X_test)

In [21]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_test,Y_pred)
print(f"Training Accuracy of Logistic Regression is {accuracy_score(Y_train, regressor.predict(X_train))}")
print(f"Test Accuracy of Logistic Regression is {accuracy}")

Training Accuracy of Logistic Regression is 0.744299674267101
Test Accuracy of Logistic Regression is 0.7987012987012987


## Step6: Predict with new data and prepare for later steps

In [17]:
testdata = pd.read_csv('testdata.csv')
testdata.head(10)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            7      156             78             42       85  33.2   
1            2      112             68             26      120  26.5   
2            5      130             82             29      132  26.2   
3            3       94             64             25       70  28.7   
4            0      120             74             18      176  29.9   
5            6      145             70             13      120  29.7   
6            1      132             78             20      141  32.5   
7            8      179             72             16      137  32.7   
8            4      158             88             28      110  37.0   
9            9      118             84             47      230  44.6   

   DiabetesPedigreeFunction  Age  
0                     0.450   54  
1                     0.730   29  
2                     0.618   35  
3                     0.560   25  
4                     0.432   28  
5                     0.550   45  
6                     0.394   21  
7                     0.719   36  
8                     0.942   27  
9                     0.816   45

In [18]:
test_pred = regressor.predict(testdata)

In [19]:
dataframe = pd.DataFrame({'outcome':test_pred})
dataframe.to_csv("test_outcome.csv", index = False)